# Import 

In [35]:
import pandas as pd
import numpy as np
from sklearn.linear_model import ElasticNet, ElasticNetCV, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn import svm
from sklearn.model_selection import KFold, cross_validate, LeaveOneOut, train_test_split, GridSearchCV
from sklearn.metrics import r2_score


# Read Data

In [2]:
data_df = pd.read_csv('./data/eig_centrality.csv', header=None)
data_df = data_df.loc[:, (data_df != 0).any(axis=0)] # drop columns with zeros
data_df.head()

,0,1,2,6,7,10,12,13,14,15,...,285,286,287,288,289,290,291,292,293,294
0,1,-0.067894,-0.062832,-0.031240,-0.071004,-0.030888,0.009534,0.044660,0.043163,0.025440,...,-0.112760,-0.106070,0.000000e+00,0.000000e+00,-0.028195,-0.024616,-0.042535,-0.087451,0.003145,-0.047196
1,1,0.031793,0.025996,0.122030,0.109220,-0.005733,0.017223,-0.072822,-0.077072,-0.126650,...,0.078364,0.049836,0.000000e+00,6.938900e-18,-0.076947,-0.078418,-0.064815,0.095082,0.054585,0.076347
2,1,-0.005251,-0.015806,-0.124040,-0.117020,-0.017355,-0.035984,0.106950,0.011851,0.077032,...,0.021125,0.014470,0.000000e+00,-1.734700e-18,0.068705,0.084321,0.047107,0.004649,-0.142760,-0.129450
3,1,0.123150,0.096576,-0.045236,-0.005143,-0.017959,-0.060064,0.008232,-0.076910,-0.042061,...,-0.043127,-0.024063,5.551100e-17,-1.110200e-16,-0.064510,-0.100380,-0.112950,-0.099919,-0.006036,-0.004228
4,1,-0.059846,-0.024500,-0.022105,0.022153,0.079495,-0.013115,0.019867,0.032033,0.012232,...,-0.026404,-0.004397,-4.336800e-19,0.000000e+00,-0.068379,0.006091,-0.050172,0.056009,0.077894,0.016500


In [39]:
# Extract data
X = abs(data_df.iloc[:,1:])
y = data_df.iloc[:,0]

print(X.shape)
print(y.shape)
print(X)

(60, 273)
(60,)
         1         2         6         7         10        12        13   \
0   0.067894  0.062832  0.031240  0.071004  0.030888  0.009534  0.044660   
1   0.031793  0.025996  0.122030  0.109220  0.005733  0.017223  0.072822   
2   0.005251  0.015806  0.124040  0.117020  0.017355  0.035984  0.106950   
3   0.123150  0.096576  0.045236  0.005143  0.017959  0.060064  0.008232   
4   0.059846  0.024500  0.022105  0.022153  0.079495  0.013115  0.019867   
5   0.047201  0.023249  0.107190  0.117740  0.010293  0.040863  0.098316   
6   0.089435  0.080275  0.001140  0.014447  0.027198  0.026102  0.116720   
7   0.092537  0.003668  0.129430  0.107730  0.088690  0.042226  0.059388   
8   0.022265  0.048711  0.048122  0.032252  0.037123  0.059852  0.011789   
9   0.111080  0.023740  0.027010  0.011543  0.089719  0.020014  0.013929   
10  0.106100  0.110970  0.030248  0.030623  0.060585  0.005032  0.020973   
11  0.034145  0.039078  0.023830  0.007204  0.005666  0.064902  0.085135

# Elastic Regression

In [4]:
ElasticNet?

Init signature:
ElasticNet(
    alpha=1.0,
    *,
    l1_ratio=0.5,
    fit_intercept=True,
    normalize='deprecated',
    precompute=False,
    max_iter=1000,
    copy_X=True,
    tol=0.0001,
    warm_start=False,
    positive=False,
    random_state=None,
    selection='cyclic',
)
Docstring:     
Linear regression with combined L1 and L2 priors as regularizer.

Minimizes the objective function::

        1 / (2 * n_samples) * ||y - Xw||^2_2
        + alpha * l1_ratio * ||w||_1
        + 0.5 * alpha * (1 - l1_ratio) * ||w||^2_2

If you are interested in controlling the L1 and L2 penalty
separately, keep in mind that this is equivalent to::

        a * ||w||_1 + 0.5 * b * ||w||_2^2

where::

        alpha = a + b and l1_ratio = a / (a + b)

The parameter l1_ratio corresponds to alpha in the glmnet R package while
alpha corresponds to the lambda parameter in glmnet. Specifically, l1_ratio
= 1 is the lasso penalty. Currently, l1_ratio <= 0.01 is not reliable,
unless you supply your own

In [5]:
regr = ElasticNet(alpha=0.0001, l1_ratio=0.5, max_iter=10000, selection="cyclic")
regr.fit(X,y)
y_pred = regr.predict(X)
print(r2_score(y,y_pred))

0.9969116241620934


In [6]:
print(regr.coef_)

[-0.          0.95863804 -0.         -0.         -0.          0.
  0.00940015  0.18521572 -0.0566415   0.         -0.2775     -0.78838429
  0.         -0.          0.00611686  0.2157303  -0.4520664  -0.
 -0.27802111 -0.         -0.          0.01963635 -0.55934513 -0.
 -0.          0.         -0.24961019 -0.          1.27379961 -1.17036357
 -1.69152178 -0.36680777  0.68040726  0.78049129  0.          1.07249191
 -0.07449389  0.          0.         -1.07217178 -0.          0.
  0.         -0.86995731  1.51313881  0.         -0.59994641 -1.20673343
  0.         -0.         -0.04946407 -0.         -0.85931877 -0.
 -0.54376361  0.         -1.19145322  0.          0.          0.
  0.         -1.3486124  -0.41283472  0.20895808  0.         -0.
 -1.2679266   0.          0.80723126  1.45790068 -0.64820979  0.778846
 -0.         -0.          0.         -0.         -0.         -0.64965506
 -0.         -0.          0.94582552 -0.          1.64807665  1.06954985
  0.          0.          0.7459858 

# Feature selector

In [7]:
regr = ElasticNet(alpha = 0.001, random_state=2021, selection='random', max_iter=10000)
selector = SelectFromModel(estimator=regr, max_features=100).fit(X,y)
len(selector.get_support()[selector.get_support()==True])

85

In [8]:
selector.get_feature_names_out()

array(['x1', 'x11', 'x16', 'x26', 'x28', 'x29', 'x30', 'x35', 'x39',
       'x43', 'x44', 'x46', 'x47', 'x52', 'x56', 'x61', 'x62', 'x66',
       'x68', 'x69', 'x70', 'x71', 'x77', 'x80', 'x82', 'x83', 'x86',
       'x95', 'x96', 'x101', 'x103', 'x104', 'x106', 'x107', 'x108',
       'x109', 'x110', 'x111', 'x112', 'x113', 'x117', 'x119', 'x120',
       'x121', 'x122', 'x123', 'x124', 'x125', 'x130', 'x132', 'x143',
       'x149', 'x152', 'x153', 'x155', 'x157', 'x158', 'x168', 'x173',
       'x178', 'x182', 'x184', 'x193', 'x194', 'x196', 'x198', 'x201',
       'x202', 'x203', 'x207', 'x211', 'x212', 'x214', 'x219', 'x221',
       'x232', 'x240', 'x247', 'x252', 'x254', 'x262', 'x263', 'x264',
       'x268', 'x269'], dtype=object)

In [9]:
X_new = selector.transform(X)
print(X_new.shape)

(60, 85)


In [10]:
# Train SVM
clf = svm.SVC(kernel='rbf', C=1000, gamma='auto')
results = cross_validate(clf, X_new, y, cv=10, return_train_score=True)

In [11]:
results

{'fit_time': array([0.00119686, 0.00051832, 0.00045681, 0.00040007, 0.00040603,
        0.00041699, 0.00037098, 0.00034881, 0.00038099, 0.00036502]),
 'score_time': array([0.00027895, 0.00015569, 0.00014997, 0.00012898, 0.00012875,
        0.00012803, 0.00012422, 0.00012493, 0.00012517, 0.00012589]),
 'test_score': array([0.83333333, 0.66666667, 1.        , 0.66666667, 1.        ,
        1.        , 1.        , 0.83333333, 1.        , 1.        ]),
 'train_score': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])}

In [12]:
np.mean(results['test_score'])

0.9

# Formalize the approach

In [13]:
# Test Train Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape)
print(X_test.shape)

(48, 273)
(12, 273)


In [14]:
# Use ElasticNet for for feature selection
regr = ElasticNet(alpha=0.0001, l1_ratio=0.5, max_iter=10000, selection="cyclic")
regr.fit(X_train,y_train)
y_regr_pred = regr.predict(X_train)
print(r2_score(y_train,y_regr_pred))

0.9979396319972256


In [15]:
# Transform data using the model
selector = SelectFromModel(estimator=regr, max_features=100).fit(X_train,y_train)
len(selector.get_support()[selector.get_support()==True])
X_train_new = selector.transform(X_train)
X_test_new = selector.transform(X_test)
print(X_train_new.shape)
print(X_test_new.shape)

(48, 87)
(12, 87)


In [16]:
# Train SVM model
clf = svm.SVC(kernel='rbf', C=1000, gamma='auto')
clf.fit(X_train_new, y_train)
print(clf.score(X_train_new, y_train))
print(clf.score(X_test_new, y_test))

1.0
0.5833333333333334


# Grid Search

In [17]:
from custom_models import ElasticSVC

In [42]:
# Set Gridsearch Params
params = {
    'alpha': [0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 0.6, 1],
    'l1_ratio': [0, 0.25, 0.5, 0.75, 1],
    'fit_intercept': [True, False],
    'svm_C': [1,10,100,1000],
    'gamma': ['scale','auto',0.01, 0.001, 0.0001],
}

In [43]:
# Initialize model and grid search
# model_pipe = ElasticSVC(max_iterations=10000)
# grid = GridSearchCV(model_pipe, cv=10, param_grid=params)

In [44]:
# Run the grid search
# grid.fit(X,y)

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.626e+00, tolerance: 1.350e-03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.575e+00, tolerance: 1.348e-03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using o

GridSearchCV(cv=10, estimator=ElasticSVC(max_iterations=10000),
             param_grid={'alpha': [0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 0.6, 1],
                         'fit_intercept': [True, False],
                         'gamma': ['scale', 'auto', 0.01, 0.001, 0.0001],
                         'l1_ratio': [0, 0.25, 0.5, 0.75, 1],
                         'svm_C': [1, 10, 100, 1000]})

In [48]:
# Sort Results
# pd.DataFrame(grid.cv_results_).sort_values('mean_test_score', ascending=False)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_fit_intercept,param_gamma,param_l1_ratio,param_svm_C,params,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
339,0.002843,0.000393,0.001088,0.000046,0.003,False,auto,1,1000,"{'alpha': 0.003, 'fit_intercept': False, 'gamm...",...,0.166667,0.833333,0.666667,0.333333,0.666667,0.666667,0.833333,0.583333,0.214087,1
319,0.002857,0.000413,0.001075,0.000038,0.003,False,scale,1,1000,"{'alpha': 0.003, 'fit_intercept': False, 'gamm...",...,0.333333,0.833333,0.666667,0.333333,0.500000,0.666667,0.666667,0.566667,0.169967,2
318,0.003012,0.000489,0.001083,0.000046,0.003,False,scale,1,100,"{'alpha': 0.003, 'fit_intercept': False, 'gamm...",...,0.333333,0.833333,0.666667,0.333333,0.500000,0.666667,0.666667,0.566667,0.169967,2
317,0.002978,0.000404,0.001135,0.000047,0.003,False,scale,1,10,"{'alpha': 0.003, 'fit_intercept': False, 'gamm...",...,0.333333,0.833333,0.666667,0.333333,0.500000,0.666667,0.666667,0.566667,0.169967,2
1439,0.002317,0.000724,0.001382,0.000685,1,True,auto,1,1000,"{'alpha': 1, 'fit_intercept': True, 'gamma': '...",...,0.333333,0.833333,0.666667,0.333333,0.333333,0.666667,0.500000,0.550000,0.167498,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947,0.002090,0.000069,0.001307,0.000044,0.1,False,0.01,0.25,1000,"{'alpha': 0.1, 'fit_intercept': False, 'gamma'...",...,0.333333,0.333333,0.333333,0.166667,0.333333,0.333333,0.333333,0.366667,0.100000,1596
738,0.002623,0.000379,0.001589,0.000256,0.03,False,auto,1,100,"{'alpha': 0.03, 'fit_intercept': False, 'gamma...",...,0.500000,0.500000,0.166667,0.166667,0.000000,0.500000,0.500000,0.366667,0.179505,1597
739,0.002058,0.000224,0.001224,0.000118,0.03,False,auto,1,1000,"{'alpha': 0.03, 'fit_intercept': False, 'gamma...",...,0.000000,0.500000,0.333333,0.166667,0.166667,0.500000,0.333333,0.350000,0.189297,1598
735,0.002519,0.000175,0.001307,0.000061,0.03,False,auto,0.75,1000,"{'alpha': 0.03, 'fit_intercept': False, 'gamma...",...,0.166667,0.166667,0.333333,0.333333,0.333333,0.333333,0.333333,0.350000,0.138444,1598


In [49]:
grid.best_params_

{'alpha': 0.003,
 'fit_intercept': False,
 'gamma': 'auto',
 'l1_ratio': 1,
 'svm_C': 1000}

In [50]:
loo = LeaveOneOut()

results = cross_validate(ElasticSVC(alpha=0.003, fit_intercept=False, gamma='auto', svm_C=1000),
                         X, y, cv=loo)
results

{'fit_time': array([0.00954819, 0.01024532, 0.0088079 , 0.00798893, 0.0082252 ,
        0.00885701, 0.00868607, 0.00928092, 0.0080409 , 0.0078218 ,
        0.00764179, 0.00907302, 0.01015997, 0.00866508, 0.00823307,
        0.00899196, 0.00792909, 0.00812197, 0.00795913, 0.00777197,
        0.008389  , 0.00774002, 0.00901794, 0.00855923, 0.0103209 ,
        0.00744867, 0.00747895, 0.0082922 , 0.00861573, 0.0079968 ,
        0.00861716, 0.00902987, 0.00889707, 0.00896001, 0.00827408,
        0.0083518 , 0.00873804, 0.00825977, 0.00830412, 0.00759697,
        0.00784183, 0.00715995, 0.00774693, 0.00792789, 0.00844789,
        0.00940633, 0.00868201, 0.00861812, 0.00792289, 0.00751209,
        0.00792217, 0.00812387, 0.00873184, 0.00891399, 0.00841808,
        0.00830889, 0.01006007, 0.00935388, 0.00939393, 0.00777197]),
 'score_time': array([0.00130391, 0.00130773, 0.00132394, 0.00110483, 0.00116992,
        0.00127101, 0.001297  , 0.00170088, 0.00109911, 0.00108099,
        0.00109124, 

In [51]:
np.mean(results['test_score'])

0.5